#### Important  Note:

In order for this to work you need to install the develop branch of pyowm, this can be done using:
```
pip install git+https://github.com/csparpa/pyowm.git@develop
```

In [1]:
import pyowm
import pandas as pd
import numpy as np
import json
import requests
from pyowm.weatherapi25.parsers import forecastparser

from auth import (
    OWM_api_key)

In [2]:
owm = pyowm.OWM(OWM_api_key)

In [3]:
park_loc = pd.DataFrame({
    'name': ['Prospect Park', 'Wooster Square'],
    'lat': [40.662551, 41.304745],
    'long': [-73.969256, -72.917757]})

In [4]:
park_name = 'Prospect Park'
park_lat = park_loc.loc[park_loc.name == park_name, 'lat'].iloc[0]
park_long = park_loc.loc[park_loc.name == park_name, 'long'].iloc[0]

park_fcstr = owm.three_hours_forecast_at_coords(
    float(park_lat),float(park_long))

In [5]:
OWM_forcast_cols = [
    'park_name',
    'reception_time',
    'reference_time',
    'sunset_time',
    'sunrise_time',
    'clouds',
    'rain',
    'snow',
    'wind_speed',
    'wind_deg',
    'humidity',
    'press',
    'press_sea',
    'temp',
    'temp_feels',
    'temp_max',
    'temp_min',
    'status',
    'detailed_status'
#     'visibility_distance',
#     'dewpoint',
#     'humidex',
#     'heat_index'
]

In [6]:
park_forcast = park_fcstr.get_forecast()
park_forcast

<pyowm.weatherapi25.forecast.Forecast - reception time=2020-04-03 16:28:59+00, interval=3h>

In [7]:
x = park_forcast.to_JSON()
json.loads(x)

{'interval': '3h',
 'reception_time': 1585931339,
 'Location': {'name': 'Brooklyn',
  'coordinates': {'lon': -73.9496, 'lat': 40.6501},
  'ID': 5110302,
  'country': 'US'},
 'weathers': [{'reference_time': 1585936800,
   'sunset_time': 0,
   'sunrise_time': 0,
   'clouds': 100,
   'rain': {'3h': 0.8},
   'snow': {},
   'wind': {'speed': 8.61, 'deg': 349},
   'humidity': 80,
   'pressure': {'press': 1005, 'sea_level': 1005},
   'temperature': {'temp': 283.04,
    'temp_kf': 0.74,
    'temp_max': 283.04,
    'temp_min': 282.3,
    'feels_like': 276.23},
   'status': 'Rain',
   'detailed_status': 'light rain',
   'weather_code': 500,
   'weather_icon_name': '10d',
   'visibility_distance': None,
   'dewpoint': None,
   'humidex': None,
   'heat_index': None},
  {'reference_time': 1585947600,
   'sunset_time': 0,
   'sunrise_time': 0,
   'clouds': 100,
   'rain': {'3h': 0.31},
   'snow': {},
   'wind': {'speed': 8.76, 'deg': 0},
   'humidity': 76,
   'pressure': {'press': 1006, 'sea_level'

In [8]:
park_forcast.get_weathers()[0].get_temperature(unit='fahrenheit')

{'temp': 49.8,
 'temp_kf': -458.34,
 'temp_max': 49.8,
 'temp_min': 48.47,
 'feels_like': 37.54}

In [9]:
def forcast_to_df(park_forcast, t_sunrise, t_sunset, tz_correct):
    ''' Takes a forcast object and sunrise/sunset time,
        Returns datafram of relevant info
    '''
    # Neet to adjust sunrise/set times, should only go
    # into columns of the same day (or just use last day in window?)
    
    forcast_df = pd.DataFrame(columns=OWM_forcast_cols)

    for x in park_forcast.get_weathers():
        if (t_sunrise == 0) or (t_sunset == 0):
            x_sunrise = np.nan
            x_sunset = np.nan
        else:
            x_sunrise = t_sunrise + tz_correct
            x_sunset = t_sunset + tz_correct

        x_rain = 0
        if x.get_rain():
            x_rain = x.get_rain()['3h']

        x_snow = 0
        if x.get_snow():
            x_snow = x.get_snow()['3h']

#         x_vis_dist = x.get_visibility_distance()
#         if x_vis_dist is None:
#             x_vis_dist = np.nan

#         x_dew = x.get_dewpoint()
#         if x_dew is None:
#             x_dew = np.nan

#         x_humid = x.get_humidex()
#         if x_humid is None:
#             x_humid = np.nan

#         x_heat_idx = x.get_heat_index()
#         if x_heat_idx is None:
#             x_heat_idx = np.nan

        to_add_dict = {
            'park_name': [park_name],
            'reception_time': [park_forcast.get_reception_time('unix') + tz_correct],
            'reference_time': [x.get_reference_time() + tz_correct],
            'sunrise_time': [x_sunrise],
            'sunset_time': [x_sunset],
            'clouds': [x.get_clouds()],
            'rain': [x_rain],
            'snow': [x_snow],
            'wind_speed': [x.get_wind(unit='meters_sec')['speed']],
            'wind_deg': [x.get_wind(unit='meters_sec')['deg']],
            'humidity': [x.get_humidity()],
            'press': [x.get_pressure()['press']],
            'press_sea': [x.get_pressure()['sea_level']],
            'temp': [x.get_temperature(unit='fahrenheit')['temp']],
            'temp_feels': [x.get_temperature(unit='fahrenheit')['feels_like']],
            'temp_max': [x.get_temperature(unit='fahrenheit')['temp_max']],
            'temp_min': [x.get_temperature(unit='fahrenheit')['temp_min']],
            'status': [x.get_status()],
            'detailed_status': [x.get_detailed_status()]
#             'visibility_distance': [x_vis_dist],
#             'dewpoint': [x_dew],
#             'humidex': [x_humid],
#             'heat_index': [x_heat_idx]
        }
        to_add = pd.DataFrame(to_add_dict)
        forcast_df = forcast_df.append(to_add, ignore_index=True)
    return forcast_df

In [10]:
forcast_df = forcast_to_df(park_forcast, 0, 0, 10)
forcast_df.head()

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,humidity,press,press_sea,temp,temp_feels,temp_max,temp_min,status,detailed_status
0,Prospect Park,1585931349,1585936810,NaN,NaN,100,0.80,0,8.61,349,80,1005,1005,49.80,37.54,49.80,48.47,Rain,light rain
1,Prospect Park,1585931349,1585947610,NaN,NaN,100,0.31,0,8.76,0,76,1006,1006,51.06,38.59,51.06,50.07,Rain,light rain
2,Prospect Park,1585931349,1585958410,NaN,NaN,100,0.18,0,8.29,6,77,1009,1009,50.65,38.75,50.65,49.98,Rain,light rain
3,Prospect Park,1585931349,1585969210,NaN,NaN,99,0.00,0,7.70,11,75,1011,1011,49.89,38.43,49.89,49.57,Clouds,overcast clouds
4,Prospect Park,1585931349,1585980010,NaN,NaN,94,0.00,0,6.33,14,76,1012,1012,48.51,38.57,48.51,48.51,Clouds,overcast clouds


In [11]:
test_weather = park_forcast.get_weathers()[0]

In [12]:
park_forcast.to_JSON()

'{"interval": "3h", "reception_time": 1585931339, "Location": {"name": "Brooklyn", "coordinates": {"lon": -73.9496, "lat": 40.6501}, "ID": 5110302, "country": "US"}, "weathers": [{"reference_time": 1585936800, "sunset_time": 0, "sunrise_time": 0, "clouds": 100, "rain": {"3h": 0.8}, "snow": {}, "wind": {"speed": 8.61, "deg": 349}, "humidity": 80, "pressure": {"press": 1005, "sea_level": 1005}, "temperature": {"temp": 283.04, "temp_kf": 0.74, "temp_max": 283.04, "temp_min": 282.3, "feels_like": 276.23}, "status": "Rain", "detailed_status": "light rain", "weather_code": 500, "weather_icon_name": "10d", "visibility_distance": null, "dewpoint": null, "humidex": null, "heat_index": null}, {"reference_time": 1585947600, "sunset_time": 0, "sunrise_time": 0, "clouds": 100, "rain": {"3h": 0.31}, "snow": {}, "wind": {"speed": 8.76, "deg": 0}, "humidity": 76, "pressure": {"press": 1006, "sea_level": 1006}, "temperature": {"temp": 283.74, "temp_kf": 0.55, "temp_max": 283.74, "temp_min": 283.19, "fe

In [13]:
json_str = '{"cod":"200","message":0,"cnt":40,"list":[{"dt":1585861200,"main":{"temp":281.88,"feels_like":275.02,"temp_min":280.27,"temp_max":281.88,"pressure":1001,"sea_level":1001,"grnd_level":1000,"humidity":76,"temp_kf":1.61},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":8.12,"deg":332},"rain":{"3h":0.42},"sys":{"pod":"d"},"dt_txt":"2020-04-02 21:00:00"},{"dt":1585872000,"main":{"temp":282.02,"feels_like":275,"temp_min":280.81,"temp_max":282.02,"pressure":1002,"sea_level":1002,"grnd_level":1002,"humidity":75,"temp_kf":1.21},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":100},"wind":{"speed":8.33,"deg":329},"rain":{"3h":0.58},"sys":{"pod":"n"},"dt_txt":"2020-04-03 00:00:00"},{"dt":1585882800,"main":{"temp":281.51,"feels_like":274.68,"temp_min":280.71,"temp_max":281.51,"pressure":1001,"sea_level":1001,"grnd_level":1001,"humidity":79,"temp_kf":0.8},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":100},"wind":{"speed":8.13,"deg":332},"rain":{"3h":0.77},"sys":{"pod":"n"},"dt_txt":"2020-04-03 03:00:00"},{"dt":1585893600,"main":{"temp":281.31,"feels_like":275.01,"temp_min":280.91,"temp_max":281.31,"pressure":1001,"sea_level":1001,"grnd_level":1000,"humidity":81,"temp_kf":0.4},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":100},"wind":{"speed":7.42,"deg":338},"rain":{"3h":0.48},"sys":{"pod":"n"},"dt_txt":"2020-04-03 06:00:00"},{"dt":1585904400,"main":{"temp":281.05,"feels_like":275.07,"temp_min":281.05,"temp_max":281.05,"pressure":1001,"sea_level":1001,"grnd_level":1000,"humidity":82,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":100},"wind":{"speed":6.94,"deg":341},"rain":{"3h":0.2},"sys":{"pod":"n"},"dt_txt":"2020-04-03 09:00:00"},{"dt":1585915200,"main":{"temp":281.04,"feels_like":275.21,"temp_min":281.04,"temp_max":281.04,"pressure":1003,"sea_level":1003,"grnd_level":1002,"humidity":83,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":6.77,"deg":347},"rain":{"3h":0.64},"sys":{"pod":"d"},"dt_txt":"2020-04-03 12:00:00"},{"dt":1585926000,"main":{"temp":282.16,"feels_like":275.7,"temp_min":282.16,"temp_max":282.16,"pressure":1004,"sea_level":1004,"grnd_level":1004,"humidity":80,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":7.84,"deg":5},"rain":{"3h":0.73},"sys":{"pod":"d"},"dt_txt":"2020-04-03 15:00:00"},{"dt":1585936800,"main":{"temp":282.4,"feels_like":275.95,"temp_min":282.4,"temp_max":282.4,"pressure":1006,"sea_level":1006,"grnd_level":1006,"humidity":80,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":7.89,"deg":11},"rain":{"3h":0.44},"sys":{"pod":"d"},"dt_txt":"2020-04-03 18:00:00"},{"dt":1585947600,"main":{"temp":282.16,"feels_like":276.23,"temp_min":282.16,"temp_max":282.16,"pressure":1008,"sea_level":1008,"grnd_level":1008,"humidity":81,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":7.14,"deg":14},"rain":{"3h":0.24},"sys":{"pod":"d"},"dt_txt":"2020-04-03 21:00:00"},{"dt":1585958400,"main":{"temp":281.67,"feels_like":276.09,"temp_min":281.67,"temp_max":281.67,"pressure":1011,"sea_level":1011,"grnd_level":1011,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":100},"wind":{"speed":6.5,"deg":16},"sys":{"pod":"n"},"dt_txt":"2020-04-04 00:00:00"},{"dt":1585969200,"main":{"temp":280.96,"feels_like":275.22,"temp_min":280.96,"temp_max":280.96,"pressure":1013,"sea_level":1013,"grnd_level":1012,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":98},"wind":{"speed":6.53,"deg":19},"sys":{"pod":"n"},"dt_txt":"2020-04-04 03:00:00"},{"dt":1585980000,"main":{"temp":280.25,"feels_like":274.92,"temp_min":280.25,"temp_max":280.25,"pressure":1014,"sea_level":1014,"grnd_level":1013,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":92},"wind":{"speed":5.75,"deg":21},"sys":{"pod":"n"},"dt_txt":"2020-04-04 06:00:00"},{"dt":1585990800,"main":{"temp":279.98,"feels_like":275.19,"temp_min":279.98,"temp_max":279.98,"pressure":1016,"sea_level":1016,"grnd_level":1015,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":95},"wind":{"speed":4.91,"deg":25},"sys":{"pod":"n"},"dt_txt":"2020-04-04 09:00:00"},{"dt":1586001600,"main":{"temp":279.95,"feels_like":275.13,"temp_min":279.95,"temp_max":279.95,"pressure":1017,"sea_level":1017,"grnd_level":1016,"humidity":80,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":88},"wind":{"speed":4.89,"deg":38},"sys":{"pod":"d"},"dt_txt":"2020-04-04 12:00:00"},{"dt":1586012400,"main":{"temp":281.97,"feels_like":277.86,"temp_min":281.97,"temp_max":281.97,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":69,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"clouds":{"all":71},"wind":{"speed":3.84,"deg":46},"sys":{"pod":"d"},"dt_txt":"2020-04-04 15:00:00"},{"dt":1586023200,"main":{"temp":282.58,"feels_like":279.89,"temp_min":282.58,"temp_max":282.58,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":66,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"clouds":{"all":80},"wind":{"speed":1.8,"deg":25},"sys":{"pod":"d"},"dt_txt":"2020-04-04 18:00:00"},{"dt":1586034000,"main":{"temp":283.39,"feels_like":281.26,"temp_min":283.39,"temp_max":283.39,"pressure":1017,"sea_level":1017,"grnd_level":1016,"humidity":63,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":91},"wind":{"speed":1.03,"deg":334},"sys":{"pod":"d"},"dt_txt":"2020-04-04 21:00:00"},{"dt":1586044800,"main":{"temp":281.67,"feels_like":279.68,"temp_min":281.67,"temp_max":281.67,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":73,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":95},"wind":{"speed":0.95,"deg":181},"sys":{"pod":"n"},"dt_txt":"2020-04-05 00:00:00"},{"dt":1586055600,"main":{"temp":280.34,"feels_like":277.25,"temp_min":280.34,"temp_max":280.34,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":79,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":95},"wind":{"speed":2.48,"deg":164},"sys":{"pod":"n"},"dt_txt":"2020-04-05 03:00:00"},{"dt":1586066400,"main":{"temp":279.81,"feels_like":276.64,"temp_min":279.81,"temp_max":279.81,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":95},"wind":{"speed":2.55,"deg":178},"sys":{"pod":"n"},"dt_txt":"2020-04-05 06:00:00"},{"dt":1586077200,"main":{"temp":279.85,"feels_like":276.96,"temp_min":279.85,"temp_max":279.85,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":81,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":100},"wind":{"speed":2.15,"deg":195},"sys":{"pod":"n"},"dt_txt":"2020-04-05 09:00:00"},{"dt":1586088000,"main":{"temp":280.28,"feels_like":277.45,"temp_min":280.28,"temp_max":280.28,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":82,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":98},"wind":{"speed":2.23,"deg":195},"sys":{"pod":"d"},"dt_txt":"2020-04-05 12:00:00"},{"dt":1586098800,"main":{"temp":283.78,"feels_like":281.47,"temp_min":283.78,"temp_max":283.78,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":71,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":100},"wind":{"speed":1.86,"deg":199},"sys":{"pod":"d"},"dt_txt":"2020-04-05 15:00:00"},{"dt":1586109600,"main":{"temp":284.8,"feels_like":282.67,"temp_min":284.8,"temp_max":284.8,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":68,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":100},"wind":{"speed":1.71,"deg":195},"sys":{"pod":"d"},"dt_txt":"2020-04-05 18:00:00"},{"dt":1586120400,"main":{"temp":284.13,"feels_like":281.95,"temp_min":284.13,"temp_max":284.13,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":76,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"clouds":{"all":100},"wind":{"speed":2.09,"deg":224},"rain":{"3h":0.16},"sys":{"pod":"d"},"dt_txt":"2020-04-05 21:00:00"},{"dt":1586131200,"main":{"temp":282.8,"feels_like":280.47,"temp_min":282.8,"temp_max":282.8,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":85,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":100},"wind":{"speed":2.42,"deg":222},"rain":{"3h":0.18},"sys":{"pod":"n"},"dt_txt":"2020-04-06 00:00:00"},{"dt":1586142000,"main":{"temp":281.34,"feels_like":279.23,"temp_min":281.34,"temp_max":281.34,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":90,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04n"}],"clouds":{"all":77},"wind":{"speed":1.91,"deg":244},"sys":{"pod":"n"},"dt_txt":"2020-04-06 03:00:00"},{"dt":1586152800,"main":{"temp":280.65,"feels_like":278.35,"temp_min":280.65,"temp_max":280.65,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":90,"temp_kf":0},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03n"}],"clouds":{"all":48},"wind":{"speed":1.96,"deg":283},"sys":{"pod":"n"},"dt_txt":"2020-04-06 06:00:00"},{"dt":1586163600,"main":{"temp":280.21,"feels_like":277.59,"temp_min":280.21,"temp_max":280.21,"pressure":1019,"sea_level":1019,"grnd_level":1018,"humidity":89,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04n"}],"clouds":{"all":81},"wind":{"speed":2.25,"deg":307},"sys":{"pod":"n"},"dt_txt":"2020-04-06 09:00:00"},{"dt":1586174400,"main":{"temp":281.32,"feels_like":278.19,"temp_min":281.32,"temp_max":281.32,"pressure":1020,"sea_level":1020,"grnd_level":1019,"humidity":75,"temp_kf":0},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"clouds":{"all":50},"wind":{"speed":2.59,"deg":326},"sys":{"pod":"d"},"dt_txt":"2020-04-06 12:00:00"},{"dt":1586185200,"main":{"temp":286.44,"feels_like":284.35,"temp_min":286.44,"temp_max":286.44,"pressure":1019,"sea_level":1019,"grnd_level":1019,"humidity":50,"temp_kf":0},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"clouds":{"all":44},"wind":{"speed":0.86,"deg":343},"sys":{"pod":"d"},"dt_txt":"2020-04-06 15:00:00"},{"dt":1586196000,"main":{"temp":288.16,"feels_like":284.95,"temp_min":288.16,"temp_max":288.16,"pressure":1018,"sea_level":1018,"grnd_level":1017,"humidity":37,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"clouds":{"all":72},"wind":{"speed":1.84,"deg":265},"sys":{"pod":"d"},"dt_txt":"2020-04-06 18:00:00"},{"dt":1586206800,"main":{"temp":287.93,"feels_like":285.24,"temp_min":287.93,"temp_max":287.93,"pressure":1016,"sea_level":1016,"grnd_level":1015,"humidity":42,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"clouds":{"all":100},"wind":{"speed":1.45,"deg":219},"sys":{"pod":"d"},"dt_txt":"2020-04-06 21:00:00"},{"dt":1586217600,"main":{"temp":283.68,"feels_like":280.02,"temp_min":283.68,"temp_max":283.68,"pressure":1016,"sea_level":1016,"grnd_level":1016,"humidity":61,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":94},"wind":{"speed":3.17,"deg":192},"sys":{"pod":"n"},"dt_txt":"2020-04-07 00:00:00"},{"dt":1586228400,"main":{"temp":282.92,"feels_like":280.88,"temp_min":282.92,"temp_max":282.92,"pressure":1016,"sea_level":1016,"grnd_level":1015,"humidity":88,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":96},"wind":{"speed":2.21,"deg":180},"rain":{"3h":1.32},"sys":{"pod":"n"},"dt_txt":"2020-04-07 03:00:00"},{"dt":1586239200,"main":{"temp":282.57,"feels_like":281.32,"temp_min":282.57,"temp_max":282.57,"pressure":1015,"sea_level":1015,"grnd_level":1014,"humidity":92,"temp_kf":0},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10n"}],"clouds":{"all":98},"wind":{"speed":1.19,"deg":167},"rain":{"3h":1.01},"sys":{"pod":"n"},"dt_txt":"2020-04-07 06:00:00"},{"dt":1586250000,"main":{"temp":282.31,"feels_like":280.82,"temp_min":282.31,"temp_max":282.31,"pressure":1014,"sea_level":1014,"grnd_level":1013,"humidity":93,"temp_kf":0},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":85},"wind":{"speed":1.49,"deg":83},"sys":{"pod":"n"},"dt_txt":"2020-04-07 09:00:00"},{"dt":1586260800,"main":{"temp":282.81,"feels_like":281.81,"temp_min":282.81,"temp_max":282.81,"pressure":1016,"sea_level":1016,"grnd_level":1015,"humidity":91,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"clouds":{"all":82},"wind":{"speed":0.86,"deg":43},"sys":{"pod":"d"},"dt_txt":"2020-04-07 12:00:00"},{"dt":1586271600,"main":{"temp":286.84,"feels_like":285.35,"temp_min":286.84,"temp_max":286.84,"pressure":1015,"sea_level":1015,"grnd_level":1015,"humidity":65,"temp_kf":0},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"clouds":{"all":61},"wind":{"speed":1.21,"deg":19},"sys":{"pod":"d"},"dt_txt":"2020-04-07 15:00:00"},{"dt":1586282400,"main":{"temp":288.86,"feels_like":286.91,"temp_min":288.86,"temp_max":288.86,"pressure":1014,"sea_level":1014,"grnd_level":1013,"humidity":49,"temp_kf":0},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"clouds":{"all":50},"wind":{"speed":1.19,"deg":296},"sys":{"pod":"d"},"dt_txt":"2020-04-07 18:00:00"}],"city":{"id":4839366,"name":"New Haven","coord":{"lat":41.3082,"lon":-72.9282},"country":"US","population":129779,"timezone":-14400,"sunrise":1585823532,"sunset":1585869461}}'

Errors in pyowm means I can't get sunrise/sunset time directly, so moving to:
- Calling the api manually for the forcast JSON
- Extracting the sunrise/sunset time
- Passing the JSON back into the pyowm parser then use the work from above

In [14]:
api_url = 'http://api.openweathermap.org/data/2.5/forecast'
payload = {
    'lat': park_lat,
    'lon': park_long,
    'appid': OWM_api_key}
r = requests.get(api_url, params=payload)

In [15]:
d = json.loads(r.text)

In [16]:
d

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1585936800,
   'main': {'temp': 283.04,
    'feels_like': 276.23,
    'temp_min': 282.3,
    'temp_max': 283.04,
    'pressure': 1005,
    'sea_level': 1005,
    'grnd_level': 1002,
    'humidity': 80,
    'temp_kf': 0.74},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 8.61, 'deg': 349},
   'rain': {'3h': 0.8},
   'sys': {'pod': 'd'},
   'dt_txt': '2020-04-03 18:00:00'},
  {'dt': 1585947600,
   'main': {'temp': 283.74,
    'feels_like': 276.81,
    'temp_min': 283.19,
    'temp_max': 283.74,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 1004,
    'humidity': 76,
    'temp_kf': 0.55},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 8.76, 'deg': 0},
   'rain': {'3h': 0.31},
   'sys': {'pod': 'd'},
   'dt_tx

In [17]:
sunrise_time = d['city']['sunrise']
sunset_time = d['city']['sunset']
tz_correct = d['city']['timezone']

In [18]:
weather_df = forcast_to_df(forecastparser.ForecastParser().parse_JSON(r.text), sunrise_time, sunset_time, tz_correct)

The dataframe `weather_df` is what I'll be adding to the log of forcast data.

In [19]:
weather_df.to_csv('example_weather_df.csv')